## Lab 4: Introduction to Reinforcement Leaerning

### Overview
In this lab you will get a small taste of the power Reinforcement Learning (RL) has to offer when it comes to robotics and control. Your task will be to use the [gymnasium](https://gymnasium.farama.org/introduction/basic_usage/) library to simulate a small MDP environment, and solve it using Tabular Q-Learning. 

Given the interdisciplinary nature of RL, the breadth of the topic can easily fill multiple courses. Here, you will be introduced to the math you need to implement Tabular Q-learning, but the reasoning behind all the steps is not explained in detail. I recommend starting at this [lecture](https://www.youtube.com/watch?v=E3f2Camj0Is&list=PLoROMvodv4rOSOPzutgyCTapiGlY2Nd8u&index=2) for those interested. Note that it takes three lectures (2-4) to arrive at the Q-learning formulation we will use here.

#### I. gym

#### II. Markov Decision Processes

#### III. Tabular Q-learning

### Assignment 
There is only one task at the bottom of the lab:
- **Question 1**: Implement Tabular Q-Learning

### I. gym

The [gym](https://gym.openai.com/docs/) library is a benchmark suite for RL provided by OpenAI. It is widely used in research due to its easy integration with python. Note that while I am using the deprecated version of gym, 

In [1]:
import gymnasium as gym
import numpy as np 


In this lab we will look at a small environment simulating a taxi driving whose goal is to pick up a passenger from one of four possible locations, and drop them off at another. Read the documentation for [Taxi-v3](https://gymnasium.farama.org/environments/toy_text/taxi/) to get familiar with the environment and understand it. Since the number of possible states is relatively small, we can find the optimal policy with a simple dynamic learning algorithm.  

Let's take a look at the env:

In [2]:
env = gym.make("Taxi-v3", render_mode="ansi").env
state, info = env.reset() 
print(f"state: {state}")
print(f"info: {info}")
print(env.render()) 

state: 406
info: {'prob': 1.0, 'action_mask': array([0, 1, 0, 0, 0, 0], dtype=int8)}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Note the `reset()` method returns the current state of the environment (state), which is an integer encoding the state using `((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination`. There is also additional info provided, which may be of relevant use to you. 

Once you reset the env, the second step is to control the agent interacting with it. We can do this with env.step(), where the input should be an integer b/w 0 and 5 as described above. Once we take our desired action with env.step(), it returns 5 items: the next state, the reward received for taking our action, two booleans that checks if the episode terminated or truncated, and additional information (check the documentation for details). 

**Note 1**: While the action should be sampled from the set of possible actions `action = env.action_space.sample(info["action_mask"])`, this may not be best when learning. Why?

**Note 2**: `terminated` is a boolean that is triggered when a terminal state is reached, whereas `truncated` is triggered if we are using a timelimit wrapper to shorten the duration of the episode. Currently, the episode length is 200, so at the final step only `terminated` would trigger. For this lab, you can simply ignore the output of truncated, and assume it will always output False.

In [3]:
env = gym.make("Taxi-v3", render_mode="ansi").env
state, info = env.reset() 
print(f"state: {state}")
print(f"info: {info}") 
print(env.render())

action = env.action_space.sample(info["action_mask"])
state, reward, terminated, truncated, info = env.step(action)
print(f"state: {state}")
print(f"reward: {reward}") 
print(f"terminated: {terminated}")
print(f"truncated: {truncated}") 
print(f"info: {info}")
print(env.render())


state: 426
info: {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


state: 446
reward: -1
terminated: False
truncated: False
info: {'prob': 1.0, 'action_mask': array([0, 1, 0, 1, 0, 0], dtype=int8)}
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)



### II. Markov Decision Processes
Markov Chains (MC) are powerful tools for modeling the dynamics of systems. What if we wanted to use an MC to model a certain game, evaluating different policies to compare them? We can just use MCs, but since we want a good metric to evaluate these policies, we need to consider taking different actions and recieving a reward signal for feedback. At some state $s_t$ in the MC, we can take an action $a_t$, and observe not only the next state $s_{t+1}$, but also the reward $r_t$ associated with that transition. This is how Markov Decision Processes (MDP) are formed i.e., they combine elements of control with the Markovian model.  

MDPs rely on the Markov Assumption:
$$
P(s_{t+1}|s_t,a_t)=P(s_{t+1}|h_t,a_t),
$$
where $h_t$ is the history of all states up to step $t$. Essentially, this says that we only need to know the current state and the action to predict the outcome. With this assumption, we can formulate an MDP $\mathcal{M}:\{\mathcal{S},\mathcal{A},P,R\}$ as a collection of a set of states, a set of actions, a probability transition function and a reward transition function respectively. 

We also need a way to express the policy:  
$$\pi: \mathcal{S}\mapsto\mathcal{A},$$
where in the determenistic case $\pi(s)=a$. Otherwise $\pi(a|s) = Pr.(a_t=a|s_t=s)$

At every state $s_t$, we receive a reward $r_t=R(s,a)$ for taking action $a_t$. By rolling out a certain policy $\pi$ in our MDP, we can collect what we call the return at step $t$:
$$
G_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \ldots.
$$
You can think of this as the total payout we will get if we run polciy $\pi$ in our given MDP, where $\gamma$ is a the discount factor we give towards rewards we receive in the future. We set $\gamma<1$ in order to give higher weight to rewards that come sooner, but also this makes sure that our return $G_t$ is finite even if our policy runs for an infinite time. This way we do not have to worry about the horizon of our MDP. 





#### Value and Q Function.
A major component in reinforcement learning is keeping track of the expected return a certain policy will provide in an environment. This is defined as the value function:
$$
V^{\pi}(s_t=s)=E_{\pi}[G_t|s_t=s].
$$
If we knew the value function for any policy $\pi$, we can easily compare the quality of different polcieis. But we can go one step further with our evaluation. Say at a given state $s_t$ we wanted to evaluate our policy $\pi$, but only after taking a certain action $a_t$, and then continuing with our policy $\pi$. This is defined as the Q-function: 
$$
Q^{\pi}(s_t=s,a_t=a) = R(s,a) + \gamma\sum_{s'\in\mathcal{S}}P(s'|s,a)V^{\pi}(s')
$$
The methodology here is that we can now evaluate our off policy decision to take action $a$. Since we know how we can estimate the value function, computing the Q-values allows us to take $\max_a Q^\pi(s,a)$ instead of following the policy, since this is gauranteed to be atleast as good as following $\pi$. 

### III. Tabular Q-Learning
In Q-Learning we measure the Temporal Difference between **target** and **current estimate** of the Q-function. 
$$
\begin{align}
\textrm{target: }&\qquad r_t + \gamma\max_{a'}Q(s_{t+1},a')\\
\textrm{estimate: }&\qquad Q(s_t,a_t)
\end{align}
$$
Their difference measures the error between what the environment is telling us, and our current estimate. We can use this error to update our current estimate, which formulates the Q-learning algorithm:
$$
Q(s_t,a_t) := Q(s_t,a_t)+\alpha\Big(r_t+\gamma\max_{a'}Q(s_{t+1},a')-Q(s_t,a_t)\Big).
$$
Now we just need a way to evaluate the policy, for which we use the epsilon-greedy approach: 
$$
\pi(a|s)=
\begin{cases}
\textrm{arg}\max_a Q^\pi(s,a)\qquad \textrm{w. prob } 1-\epsilon,\\
\textrm{take a random action}\qquad \textrm{w. prob } \epsilon.
\end{cases}
$$
Note how $\epsilon$ controls how much we $\textit{explore}$ the environment versus how much we $\textit{exploit}$ our policy. A lot of RL is posed as a battle between exploration and exploitation, hence $\epsilon$ is an important parameter. Typically we want to decrease the amount we explore as our policy gets better, and we do this by adapting some sort of **decay** to $\epsilon$. In addition, since $\alpha$ controls the speed at which we update our policy, you can consider adapting some decay as well. 

I will leave this element for you to experiment with, as it serves as an interesting challenge to try and find the correct parameters and fine tune. As a reference, you should be able to find the policy within 20,000 episodes (where episodes end when the taxi drops off the passenger.) In fact, even 5,000 is enough but you will see slight improvement following. 



### Question 1: Implement Tabular Q-Learning

The actual algorithm can be written within one cell, or broken up into smaller components, it is your choice. There are only two things I require:

(1) **Represent your Q-table as 2-dim numpy array**, over the state and action space. While this is not a hard requirement, it will be useful, and hence is set up in the next cell.

(2) **Plot the reward of your policy (for an episode) vs the number of episodes used during training so far**. This will be your main evaluation during training, so you should structure your code to record this metric, and plot it after each run. Does your policy converge to some reasonable reward?

(3) **Evaluate your final policy over 1000 episodes, and print the mean and standard deviation of the episodic return.**

In [4]:
# Initialize Q-table
q_table = np.random.randn(env.observation_space.n, env.action_space.n)

In [5]:
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
# colors used
ORANGE = '#FF9132'
TEAL = '#0598B0'
GREEN = '#008F00'
PURPLE = '#8A2BE2'
GRAY = '#969696'
FIG_WIDTH = 4
FIG_HEIGHT = 6

plt.rcParams.update({
    "text.usetex": False,
    "font.size": 10,
    "axes.titlesize": 10,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "lines.linewidth": 2
})

# plot


In [6]:
# evaluate your policy